# Introduction
In this example, we will build a multi-label text classifier to predict the subject areas of arXiv papers from their abstract bodies. This type of classifier can be useful for conference submission portals like OpenReview. Given a paper abstract, the portal could provide suggestions for which areas the paper would best belong to.

The dataset was collected using the arXiv Python library that provides a wrapper around the original arXiv API. Additionally, you can also find the dataset on Kaggle.

In [1]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from ast import literal_eval

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### EDA

In [14]:
arxiv_data = pd.read_csv(
    "https://github.com/soumik12345/multi-label-text-classification/releases/download/v0.2/arxiv_data.csv"
)

print(f"There are {len(arxiv_data)} rows in the dataset.")

arxiv_data.head()

There are 51774 rows in the dataset.


,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


#### Remove Duplicated records

1. Remove row-level duplicates
2. Remove records with same title

In [15]:
print('Total duplicated entries : ', arxiv_data.shape[0] - arxiv_data.drop_duplicates().shape[0])

Total duplicated entries :  12783


In [16]:
arxiv_data = arxiv_data.drop_duplicates()
print('Data volume after removing duplicates : ', arxiv_data.shape[0])

Data volume after removing duplicates :  38991


In [17]:
arxiv_data = arxiv_data[~arxiv_data["titles"].duplicated()]
print(f"There are {len(arxiv_data)} rows in the final deduplicated dataset.")

There are 38972 rows in the final deduplicated dataset.


#### Analyse terms

In [18]:
arxiv_data['terms'].unique()

array(["['cs.CV', 'cs.LG']", "['cs.CV', 'cs.AI', 'cs.LG']",
       "['cs.CV', 'cs.AI']", ..., "['stat.ML', 'cs.LG', 'I.2.1; J.3']",
       "['cs.LG', 'cs.NE', 'q-bio.BM', 'stat.ML']",
       "['stat.ML', 'cs.CV', 'cs.LG', 'q-bio.QM']"], dtype=object)

In [19]:
arxiv_data['terms'].value_counts()

['cs.CV']                                          12747
['cs.LG', 'stat.ML']                                4074
['cs.LG']                                           2046
['cs.CV', 'cs.LG']                                  1486
['cs.LG', 'cs.AI']                                  1206
                                                   ...  
['cs.LG', 'cs.CL', 'cs.HC', 'stat.ML']                 1
['cs.LG', 'cs.AI', 'cs.CL', 'cs.PL', 'stat.ML']        1
['cs.LG', 'cs.CL', 'stat.ME', 'stat.ML']               1
['cs.LG', 'cs.CL', 'cs.LO', 'stat.ML']                 1
['stat.ML', 'cs.CV', 'cs.LG', 'q-bio.QM']              1
Name: terms, Length: 3157, dtype: int64

Out of 3157 distinct labels, 2321 labels occur only once. In order to better prepare the data for classification and the problem itself, we should remove these entries

In [22]:
arxiv_data.groupby("terms").filter(lambda x: len(x) == 1).shape[0]

2321

In [23]:
# Filtering the rare terms
arxiv_data_filtered = arxiv_data.groupby("terms").filter(lambda x: len(x) > 1)
arxiv_data_filtered.shape

(36651, 3)